In [ ]:
# Initialize pyegeria

import os
view_server = os.environ.get("EGERIA_VIEW_SERVER","qs-view-server")
url = os.environ.get("EGERIA_VIEW_SERVER_URL","https://host.docker.internal:9443")
user_id = os.environ.get("EGERIA_USER", "peterprofile")
user_pwd = os.environ.get("EGERIA_USER_PASSWORD")
egeria_width = 150

from pyegeria import EgeriaTech, EgeriaCat, TEMPLATE_GUIDS, INTEGRATION_GUIDS
import asyncio
import nest_asyncio
nest_asyncio.apply()

from pyegeria import load_mermaid, render_mermaid, generate_process_graph
load_mermaid()

from pyegeria.commands.ops.monitor_integ_daemon_status import display_integration_daemon_status
from pyegeria.commands.ops.monitor_engine_activity_c import display_engine_activity_c
from pyegeria.commands.ops.monitor_gov_eng_status import display_gov_eng_status

In [ ]:
# print functions

def print_property(indent, property_name, property_value):
    print(indent + property_name + ": " + property_value)

def get_property(indent, property_name, properties):
    if properties:
       property_value=properties.get(property_name)
       if property_value:
           print_property(indent, property_name, property_value)

def print_element_header(indent, element_header):
    type_name = element_header["type"]["typeName"]
    guid = element_header["guid"]
    print(indent + type_name + " [" + guid + "]")

def print_properties(indent, property_name, properties):
    if properties:
        if type(properties) is str:
            print_property(indent, property_name, properties)
        elif type(properties) is int:
            print_property(indent, property_name, "{}".format(properties))
        elif type(properties) is dict:
            for key in properties.keys():
                print_properties(indent, key, properties[key])
        else:
            print(f"Funny property: type is {type(properties)}")
            
def print_element(indent, element):
    if element:
        print()
        print_element_header(indent + "", element.get("elementHeader"))
        print_properties(indent + " > ", "properties", element.get("properties"))
        print_properties(indent + " > ", "referenceableProperties", element.get("referenceableProperties"))

def print_related_elements(indent, related_elements):
    if related_elements:
        for related_element in related_elements:
            if related_element:
                print()
                print_element_header(indent, related_element.get("relationshipHeader"))
                print_properties(indent + " > ", "relationshipProperties", related_element.get("relationshipProperties"))
                print_element(indent + "   ", related_element.get("relatedElement"))
                                
def print_search_results(search_results):
    if search_results:
        if search_results == "no assets found":
            print(search_results)
        else:
            for asset in search_results:
                if asset:
                    print()
                    print("Asset: ")
                    print_element_header(" > " , asset.get("elementHeader"))
                    print_properties(" > ", "properties", asset.get("properties"))
                    matchingElements = asset.get("matchingElements")
                    if matchingElements:
                        for matchingElement in matchingElements:
                            print("Matching Element:")
                            print_element_header("   > " , matchingElement.get("elementHeader"))
                            print_properties("   > " , "properties", matchingElement.get("properties"))

def print_external_id_map(catalog):
    if catalog:
        catalog_guid = catalog["elementHeader"]["guid"]
        print_element("", catalog)
        external_ids = egeria_tech.get_related_elements(catalog_guid, "ExternalIdScope")
        if external_ids:
            for external_id in external_ids:
                print()
                relationship_properties = external_id.get("relationshipProperties")
                if relationship_properties:
                    print_property(" > ", "Permitted Synchronization", relationship_properties.get("permittedSynchronization"))
                related_element = external_id.get("relatedElement")
                external_id_guid = related_element["elementHeader"]["guid"]
                open_metadata_elements = egeria_tech.get_related_elements(external_id_guid, "ExternalIdLink")
                print_related_elements("      ", open_metadata_elements)


def print_governance_action_process_graph(process_graph):
    if process_graph:
        mermaid_graph = process_graph.get('mermaidGraph')
        if mermaid_graph:
            render_mermaid(mermaid_graph)
        process = process_graph.get('governanceActionProcess')
        if process:
            mermaid_specification = process.get('mermaidSpecification')
            if mermaid_specification:
                render_mermaid(mermaid_specification)

